In [101]:
import torch 
import random
import pandas as pd

import torch.nn as nn
import torch.optim as optim

torch.manual_seed(2021)

from torchtext import data 
from torchtext.legacy.data import Field, TabularDataset, LabelField, BucketIterator

from sklearn.metrics import accuracy_score

In [82]:
CONTEXT = Field(tokenize='spacy',batch_first=True,include_lengths=True)
TEXT = Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = LabelField(dtype = torch.float,batch_first=True)

C:\Users\Dunja\anaconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [102]:
fields = {'label':('label', LABEL), 'response':('text', TEXT)}
training_data=TabularDataset(path = 'sarcasm_detection_shared_task_twitter_training.jsonl', format = 'json', fields = fields,)
train_data, valid_data = training_data.split(split_ratio=0.9, random_state = random.seed(2021))

In [103]:
#initialize glove embeddings
TEXT.build_vocab(train_data, min_freq=1,vectors = "glove.twitter.27B.100d")  
LABEL.build_vocab(train_data,)

In [104]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
BATCH_SIZE = 16
device = 'cpu'

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)


In [105]:
class Classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, weights,
                 bidirectional, dropout):
        
        super().__init__()          
        self.embedding = nn.Embedding.from_pretrained(weights)
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        # print(type(text))
        embedded = self.embedding(text) # embedded = [batch size, sent_len, emb dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded) #hidden = [num layers * num directions, batch size, hid dim]
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1) #hidden = [batch size, hid dim * num directions]
        
        dense_outputs=self.fc(hidden)
        outputs=self.act(dense_outputs)

        # dense_outputs=self.fc(hidden)
        # outputs=self.act(hidden)
        
        return outputs

In [106]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 300
num_output_nodes = 1
num_layers = 3
bidirection = True
dropout = 0.2

pretrained_embeddings = TEXT.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings)

#instantiate the model
model = Classifier(size_of_vocab, embedding_dim, num_hidden_nodes ,num_output_nodes, num_layers, pretrained_embeddings, bidirectional=True, dropout=dropout)

#define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [107]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()  
    
    for batch in iterator:
        optimizer.zero_grad()   
        text, text_lengths = batch.text 

        predictions = model(text, text_lengths)
        loss = criterion(predictions.squeeze(), batch.label.squeeze())
        acc = accuracy_score(predictions.detach().squeeze().round().numpy(), batch.label.detach().numpy() ) 
        
        loss.backward()       
        optimizer.step()      
        
        epoch_loss += loss.item()  
        epoch_acc += acc    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [108]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    
    with torch.no_grad():  
        for batch in iterator:
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths)
            loss = criterion(predictions.squeeze(), batch.label.squeeze())
            acc = accuracy_score(predictions.detach().squeeze().round().numpy(), batch.label.detach().numpy() ) 
            
            epoch_loss += loss.item()
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [109]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        print(f"\nSaved {epoch+1}")
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f"\nEpoch {epoch+1}")
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal.  Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Saved 1

Epoch 1
	Train Loss: 0.599 | Train Acc: 66.69%
	Val.  Loss: 0.564 |  Val. Acc: 70.70%

Saved 2

Epoch 2
	Train Loss: 0.530 | Train Acc: 73.52%
	Val.  Loss: 0.553 |  Val. Acc: 70.31%

Saved 3

Epoch 3
	Train Loss: 0.507 | Train Acc: 74.38%
	Val.  Loss: 0.535 |  Val. Acc: 72.07%

Epoch 4
	Train Loss: 0.492 | Train Acc: 75.33%
	Val.  Loss: 0.545 |  Val. Acc: 73.05%

Saved 5

Epoch 5
	Train Loss: 0.486 | Train Acc: 75.40%
	Val.  Loss: 0.532 |  Val. Acc: 72.85%


In [110]:
#load weights
path='saved_weights.pt'
model.load_state_dict(torch.load(path))
model.eval()

#inference 
import spacy
nlp = spacy.load('en_core_web_sm')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  # tokenize the sentence 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          # convert to integer sequence
    length = [len(indexed)]                                    # compute no. of words
    t = torch.LongTensor(indexed).to(device)                   # convert to tensor
    t = t.unsqueeze(1).T                                       # reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   # convert to tensor
    prediction = model(t, length_tensor)                       # prediction 
    return prediction.item()               

In [111]:
twitter = pd.read_json("sarcasm_detection_shared_task_twitter_testing.jsonl", lines=True)
input_labels = [ 1 if x!='SARCASM' else 0 for x in twitter.label ]

In [112]:
outs=[]
for s in twitter.response:
    outs.append(round(predict(model,s)))

In [115]:
from sklearn.metrics import f1_score
f1 = f1_score(input_labels, outs, average='macro')
f1

0.6889510489510489

In [117]:
from sklearn import metrics
print("Precision:",metrics.precision_score(input_labels, outs))
precision =metrics.precision_score(input_labels, outs)

# Model Recall: what perce
# ntage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(input_labels, outs))
recall = metrics.recall_score(input_labels, outs)

print(f1_score(input_labels, outs, average='macro'))

Precision: 0.7293333333333333
Recall: 0.6077777777777778
0.6889510489510489
